In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import plotly.express as px
from DBSCANSupport import *

Load gravity data

In [2]:
src_dir = '/Users/martinhawks/Code/Git_repos/seamount_picking/data/'  # Update to local machine
grav = pd.read_csv(src_dir + 'test_grav.csv').drop(columns=["old_ind"])


In [3]:
longg = list(grav['Longitude'])
lat = list(grav['Latitude'])
inten = list(grav['Intensity'])
#fig.show()

In [4]:
X = grav[['Longitude', 'Latitude', 'Intensity']].to_numpy()
X

array([[-97.99166667,  -1.50833333,  -5.5733223 ],
       [-97.975     ,  -1.50833333,  -5.16694736],
       [-97.95833333,  -1.50833333,  -4.97332668],
       ...,
       [-90.04166667,  -5.99166667,  -2.35047531],
       [-90.025     ,  -5.99166667,  -0.47824201],
       [-90.00833333,  -5.99166667,  -0.67400765]])

In [5]:
#scale_x = StandardScaler().fit_transform(X)
scale_x = X

In [6]:
test_eps = np.linspace(0.5, 1, 5)
test_samp = np.arange(20, 25)


In [7]:
score, params, data_out  = gridSearch(test_eps, test_samp, scale_x, seamountDeviation, verbose=True)

Score for 0.5 and 20 is -1.119205298013245
0.5 and 21 produced too few clusters
0.5 and 22 produced too few clusters
Score for 0.5 and 23 is -2.5231788079470197
Score for 0.5 and 24 is -2.423841059602649
Score for 0.625 and 20 is -0.6225165562913907
Score for 0.625 and 21 is -0.6026490066225165
Score for 0.625 and 22 is -0.33112582781456956
Score for 0.625 and 23 is -0.32450331125827814
Score for 0.625 and 24 is -0.5364238410596026
0.75 and 20 produced too few clusters
Score for 0.75 and 21 is -0.17880794701986755
Score for 0.75 and 22 is -0.1456953642384106
Score for 0.75 and 23 is -0.1456953642384106
Score for 0.75 and 24 is -0.1390728476821192
Score for 0.875 and 20 is -0.10596026490066225
0.875 and 21 produced too few clusters
0.875 and 22 produced too few clusters
0.875 and 23 produced too few clusters
0.875 and 24 produced too few clusters
Score for 1.0 and 20 is -0.26490066225165565
Score for 1.0 and 21 is -0.423841059602649
Score for 1.0 and 22 is -0.2781456953642384
Score for 

In [9]:
score

-0.10596026490066225

In [27]:
data_out

array([[-97.99166667,  -1.50833333,   0.        ,  -5.5733223 ],
       [-97.975     ,  -1.50833333,   0.        ,  -5.16694736],
       [-97.95833333,  -1.50833333,   0.        ,  -4.97332668],
       ...,
       [-90.04166667,  -5.99166667,  -1.        ,  -2.35047531],
       [-90.025     ,  -5.99166667,  -1.        ,  -0.47824201],
       [-90.00833333,  -5.99166667,  -1.        ,  -0.67400765]])

In [28]:
cat_mounts = data_out[data_out[:, 2] == 1]
cat_mounts

array([[-90.64166667,  -1.50833333,   1.        , -67.42951202],
       [-90.925     ,  -1.525     ,   1.        , -67.75108337],
       [-90.90833333,  -1.54166667,   1.        , -67.34489441],
       [-90.84166667,  -1.55833333,   1.        , -67.75315857],
       [-90.80833333,  -1.575     ,   1.        , -67.92209625],
       [-90.64166667,  -1.575     ,   1.        , -68.17828369],
       [-90.775     ,  -1.59166667,   1.        , -67.34020996],
       [-90.64166667,  -1.59166667,   1.        , -67.38854218],
       [-90.74166667,  -1.60833333,   1.        , -66.94778442],
       [-90.725     ,  -1.60833333,   1.        , -67.95191193],
       [-90.64166667,  -1.60833333,   1.        , -67.20214844],
       [-90.70833333,  -1.625     ,   1.        , -67.33777618],
       [-90.64166667,  -1.625     ,   1.        , -67.59481049],
       [-90.675     ,  -1.64166667,   1.        , -67.34012604],
       [-90.65833333,  -1.64166667,   1.        , -67.56185913],
       [-90.64166667,  -1

In [29]:

fig = px.scatter(x=cat_mounts[:, 0], y=cat_mounts[:, 1])
fig.show()

In [25]:
test_zone=(-6, -1.5, -98, -90)
seamounts = pd.read_excel("/Users/martinhawks/Code/Git_repos/seamount_picking/data/sample_mask.txt.xlsx", \
                              sheet_name="new mask")
seamounts = seamounts.drop(columns=["VGG Height", "Radius", "base_depth", "-",
                                        "Name", "Charted", "surface_depth"])
seamounts = seamounts[(seamounts["Latitude"] >= test_zone[0]) & (seamounts["Latitude"] <= test_zone[1]) &
                          (seamounts["Longitude"] >= test_zone[2]) & (seamounts["Longitude"] <= test_zone[3])]
seamounts = seamounts.to_numpy()
fig3 = px.scatter(x=seamounts[:, 0], y=seamounts[:, 1])
fig3.show()